In [92]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
import datetime
import re
import math
abg = pd.read_csv('/tmp/out.csv', dtype={"match-scores-csv": str})

abg.sort_values("match-completed-at", inplace=True)
abg.sort_values("player1-name", inplace=True)

#abg.rename(columns = {'player1-display-name-with-invitation-email-address':'player1-name','player2-display-name-with-invitation-email-address': 'player2-name'}, inplace=True)

f = abg["match-scores-csv"].apply(lambda x: pd.Series(str(x).split("-")))
abg["player1-score"] = f[0]
abg["player1-score"] = pd.to_numeric(abg["player1-score"], errors='coerce')
abg["player2-score"] = f[1]
abg["player2-score"] = pd.to_numeric(abg["player2-score"], errors='coerce')

abg["match-length"] = abg[["player1-score", "player2-score"]].apply(max, axis=1)
#abg["player1-win_chance"] = 


#elo = pd.DataFrame({}, columns=['player1-name', 'player2-name', 'player1-id', 'player2-id'])


p1 = abg.ix[:,['player1-name']].rename(columns={"player1-name": "player_name"})
p2 = abg.ix[:,['player2-name']].rename(columns={"player2-name": "player_name"})

p1.reset_index(inplace=True,drop=True)
p2.reset_index(inplace=True,drop=True)


players = p1.append(p2)
players.drop_duplicates(inplace=True)

players["ELO"] = 1500
players["last_updated"] = None
players["last_updated"] = pd.to_datetime(players["last_updated"])
#players.set_index('player_name', drop=False, inplace=True)
#players.reindex()

players.to_csv(open("/tmp/players.csv", "w"))


#elo.unique["player_id"].unique()
#elo.head()





#abg.head(40)


In [100]:

##Elo calc


class ELO:
    
    def get_win_chance(elo1, elo2, length):
        d = elo1 - elo2
        exp = -1*(elo1-elo2) * np.sqrt(length)/2000
        return 1/(1+10**exp)
        
    def compute_change(elo_winner, elo_loser, length):
        
        points = 4 * np.sqrt(length)
        winner_change = (1 - ELO.get_win_chance(elo_winner, elo_loser, length)) * points
        loser_change = points * ELO.get_win_chance(elo_loser, elo_winner, length)
        return (winner_change, loser_change * -1)


def set_elo(row):
    
    try:
        length = row["match-length"]
    except ValueError:
        print("I am afraid {} is not a number".format(row["match-length"]))
        return pd.Series([0,0])
    
    if (math.isnan(length)):
        print("I am afraid {} is not a number".format(row["match-length"]))
        return pd.Series([0,0])
    
    
    _players = {"player1": {}, "player2": {}}
    for k in row.keys():
        m = re.search("(player[1-2])-([a-zA-Z0-9\-_]+)", k)
        if (m):
            _players[m.group(1)][m.group(2)] = row[k]
    player1 = _players["player1"]
    player2 = _players["player2"]
    
    #figure out who is the winner
   
    if player1["score"] > player2["score"]:
        winner = player1
        loser = player2
    else:
        winner = player2
        loser = player1

   
    winner["elo"] = find_by_name(winner["name"])["ELO"]
    loser["elo"] = find_by_name(loser["name"])["ELO"]
    
    elos = ELO.compute_change(int(winner["elo"]),int(loser["elo"]), length)
    
    change_player_elo(winner["name"], winner["elo"]+elos[0], row["match-completed-at"])
    change_player_elo(loser["name"], loser["elo"]+elos[1], row["match-completed-at"])

#     pp(elos)
#     pp(length)
#     pp(winner["elo"])
#     pp(loser["elo"])
    return pd.Series([elos[0],elos[1]])

def change_player_elo(player_name, new_elo, date):
    players.loc[find_by_name(player_name).index,"ELO"] = new_elo
    #players.loc[find_by_name(player_name).index,"last_updated"] = date

    
def find_by_name(name):
    return players[players['player_name'].str.contains(name)]
    
## Elo calc
abg["winner_ELO"] = 1500
abg["loser_ELO"] = 1500
abg['winchance'] = .5
abg["winner_ELO_change"] = 0
abg["loser_ELO_change"] = 0

abg[['winner_ELO_change','loser_ELO_change']] = abg.apply(set_elo, axis=1)

players.sort_values("ELO", inplace=True, ascending=False)
players.head(100)



I am afraid nan is not a number
I am afraid nan is not a number
I am afraid nan is not a number
I am afraid nan is not a number
I am afraid nan is not a number


,player_name,ELO,last_updated
783,Nathan Parker,1641.588849,NaT
861,Paul Whitter,1638.982752,NaT
206,Chirita Iulian,1629.933837,NaT
91,Ammar King,1618.934778,NaT
391,Jacob Singh,1611.367674,NaT
110,Anita Marholm,1594.319802,NaT
39,Alan Draper,1587.888849,NaT
1028,Scooter Nelson,1584.101533,NaT
468,Julia Hayward,1582.806756,NaT
506,Lisa Hewer,1581.596377,NaT


In [101]:
find_by_name("Jay")

,player_name,ELO,last_updated
432,Jay O'Sullivan,1537.402474,NaT
411,Jay O Sullivan,1531.755973,NaT
434,Jay Sull,1449.109730,NaT
